In [2]:
from src.data.create_dataset import *

In [14]:
df = read_data('2022', '02')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007284 entries, 0 to 1007283
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               936045 non-null   float64       
 1   tpep_pickup_datetime   1007284 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  1007284 non-null  datetime64[ns]
 3   passenger_count        936045 non-null   float64       
 4   trip_distance          1007284 non-null  float64       
 5   RatecodeID             936045 non-null   float64       
 6   store_and_fwd_flag     936045 non-null   object        
 7   PULocationID           1007284 non-null  int64         
 8   DOLocationID           1007284 non-null  int64         
 9   payment_type           936045 non-null   float64       
 10  fare_amount            1007284 non-null  float64       
 11  extra                  1007284 non-null  float64       
 12  mta_tax                10072

In [4]:
## Preprocess (load data -> clean data -> save preprocessed data to cloud)
!python ../main/preprocess.py

In [11]:
## Generate the features (generate features -> save the new dataframe with features to cloud)
!python ../main/feature_engineering.py

Current version: 20220506-235733


In [ ]:
## train the model (train the model -> save the model to cloud)
!python ../main/train_model.py